# Housing Regression Maching Learning Project

### Imports (Data and Packages)

In [33]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from data_preprocessing_final import cleaning, dummify, scale_data, ord_encoding, initiate_data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
%matplotlib inline 

In [34]:
housing = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)
housing = housing[housing['SaleCondition'] == 'Normal']
train_data_linear, train_target_linear, test_data_linear, test_target_linear, train_data_tree, train_target_tree, test_data_tree, test_target_tree = initiate_data(housing)


### Lasso Regression

In [35]:
## Working on Lasso Regression
from sklearn.linear_model import Lasso
lasso = Lasso()

In [36]:
alphaRange = np.linspace(0,0.002,5)
params = {'alpha': alphaRange, 'max_iter' : [10000]}

In [37]:
gs = GridSearchCV(lasso, params, scoring = 'r2')
gs.fit(train_data_linear, train_target_linear)

C:\Users\jackc\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.112356297153616, tolerance: 0.02106932755585719
  model = cd_fast.enet_coordinate_descent(
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimato

GridSearchCV(estimator=Lasso(),
             param_grid={'alpha': array([0.    , 0.0005, 0.001 , 0.0015, 0.002 ]),
                         'max_iter': [10000]},
             scoring='r2')

In [38]:
gs.best_params_

{'alpha': 0.0, 'max_iter': 10000}

In [39]:
gs.score(train_data_linear, train_target_linear)

0.9575532398240968

In [40]:
gs.score(test_data_linear, test_target_linear)


0.9262687394260829

In [41]:
predict = gs.predict(test_data_linear)
predict = pd.DataFrame(predict)
predict.columns = ['Prediction']

In [43]:
final = pd.concat([predict, test_data_linear, test_target_linear], axis =1)

In [44]:
pd.options.display.float_format = '${:,.2f}'.format
final['SalePrice'] = np.exp(final['SalePrice'])
final['Prediction'] = np.exp(final['Prediction'])
final['Pct_diff'] = ((final['Prediction'] - final['SalePrice'])/ final['SalePrice'])*100
final[['Prediction', 'PID','SalePrice', 'Pct_diff']]


,Prediction,PID,SalePrice,Pct_diff
0,"$84,066.19",902135020,"$93,000.00",$-9.61
1,"$104,691.23",535353130,"$112,500.00",$-6.94
2,"$152,950.24",527165020,"$160,000.00",$-4.41
3,"$111,632.02",535375130,"$109,000.00",$2.41
4,"$170,457.81",535152150,"$165,500.00",$3.00
...,...,...,...,...
478,"$168,843.82",526351100,"$157,000.00",$7.54
479,"$240,591.22",904100140,"$244,400.00",$-1.56
480,"$160,758.10",909254010,"$137,000.00",$17.34
481,"$195,568.69",532477030,"$192,000.00",$1.86


In [49]:
final['Pct_diff'].mean()

0.6735727763406397

## Random Forrest   

In [12]:
rf_model = RandomForestRegressor(random_state=0)
rf_model.fit(train_data_tree, train_target_tree)


RandomForestRegressor(random_state=0)

In [15]:
train_preds = rf_model.predict(train_data_tree)
train_mse = mean_squared_error(train_target_tree, train_preds)
print('Train set MSE is {:.3f}'.format(train_mse))
print('Train set R2 is {:.3f}'.format(rf_model.score(train_data_tree, train_target_tree)))

train_preds_test = rf_model.predict(test_data_tree)
test_mse = mean_squared_error(test_target_tree, train_preds_test)
print('Train set MSE is {:.3f}'.format(test_mse))
print('Train set R2 is {:.3f}'.format(rf_model.score(test_data_tree, test_target_tree)))

Train set MSE is 0.002
Train set R2 is 0.986
Train set MSE is 0.012
Train set R2 is 0.916
